# Import dataset

In [1]:
import re, spacy, pandas as pd, numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("dataset.csv")
dff = pd.read_csv("dataset2.csv", 
                  usecols = ['questions','answers'])
dff.columns = ['question', 'answer']

In [3]:
df.style.set_properties(subset=['question', 'answer'], **{'width': '300px'})

,question,answer
0,"It is really great to see exponential growth of AppliedAI. Team's efforts to consider each and every concern, and work on it is very much appreciable. Very soon AppliedAI will be one of top most and successful teaching platform.",Thank you for the kind words and the trusting us. We make sure every student who has trusted us will succeed through our course.
1,"I also want to make a startup like you in the AI field, this course give me a good start for ml and deep learning. Till the end of my B.tech ,I want to gain as much as knowledge as possible, or i can say as impossible.Now, finished my first year. Fully passionate for completing the case studies.","Learn as much as you can from this course and after your 2nd year onwards, start internships and by the time you sit for placements in your final year, you'll get as much exposure. In case if you are interested in product development companies, along with ML, also gain strong command on CS fundamentals like Data Structures & Algorithms, Design Patterns, Operating Systems, Computer Networks, DBMS. Regarding the start up plans, once if you have some experience of around 3-5 years in the corporate, you can think of a new startup because in order to run a startup along with the technical stuff, you also need management skills."
2,"sir, thanks for wonderful suggestions. But What you suggest me regarding internships, I want to get some real world exposure by doing internship in these summer vacations, I completed 10 to 11 assinments, Now In 10 to 15 days I 'll complete all course content, just case studies 'll left after that. I have full june month for completing some case studies and refresh concepts about machine learning then till the end of june ,I'll apply for internsip. Is this good idea.","Make sure your internship doesn't clash with your classes in college. Yes you can start applying for internship once if you finish the course. Once if you finish all the course content, there are chances for you to crack internships in top IT companies"
3,What are design patterns?,https://en.wikipedia.org/wiki/Software_design_pattern & https://www.geeksforgeeks.org/design-patterns-set-1-introduction/
4,Hi first of all thank you for great course Can you please provide some modules for reinforcement learning.,"Adding more content to the current course would make it quite bulky for most of our students to finish and stay motivated through the course. Hence, we did not add Reinforcement Learning (RL) to it as of now. We do not want out our students to be overwhelmed by the course content and workload. While we want to add more content, we don't want to overburden the students. We will probably create a new course (or) add RL to this course itself in the future (no guarantees) as optional topics. It should be easy for any of our students to pick up RL once they finish this course as they have all of the background in Maths and Deep-Learning. Please note that no course can cover everything in ML. Often the content in this course itself is split across 2-3 courses. But we designed the course such that students can pick up new concepts fairly easily at the end of the course. Actually, some of our students have picked up RL at the end of this course fairly easily."
5,Hi Then I suggest please add another course for RL. I joined course not due to lack of material for ML online but due to organization and flow of course and the way you teach. So I request you to please consider this request seriously. I will happily pay for another course from you.,We will surely inform you once we have the Reinforcement course ready (or) when we add RL as an optional content for this course.
6,"Hi team, Thank you for your all efforts taken for designing such great content. Especially Varma sir's way of teaching and explaining difficult concepts in a very easy-to-understand manner is praiseworthy. I have been doing this course for 4 months now and from my experience, I can say that your hard work ref

In [4]:
df.isnull().values.any()

False

# Please give suggestions on this topics

In [5]:
# Remove the rows which have answers like below
# I think removing all the links from the answers is important preprocessing.
df.iloc[3].to_list()

['What are design patterns?',
 'https://en.wikipedia.org/wiki/Software_design_pattern\n& https://www.geeksforgeeks.org/design-patterns-set-1-introduction/']

In [6]:
# Let the Capital Letter words as it is.
# or we can create a new character before word which is completely in capital letters like AMD, RAM in the 
# question below. Students writing these words in Capital letters ---correletaed---> their collective meaning
df.iloc[102].to_list()

['Hello, My system configuration is \nProcessor - AMD A8 with R5 graphics,\nRAM - 4GB\nIf i just increase ram to 8GB would configuration be fine to go ahead? \nThanks',
 'yes it will be enough. And also we can always use Google Colab which you will learn in future videos']

# Removing Answeres with only links

In [7]:
df1 = df.copy()

In [8]:
# Delete all the HTML tags like "< anyword >" ---> https://stackoverflow.com/a/12982689
i = 0
while True:
    for index, preprocessed_text in enumerate(df.iloc[:,i].to_list()):
        preprocessed_text = re.sub(r'(https?://[^\s]+)', ' ', preprocessed_text).strip()
        preprocessed_text = re.sub('[^A-Za-z0-9@.?:_/\'-]', ' ', preprocessed_text)
        preprocessed_text = re.sub('\s+', ' ', preprocessed_text).strip() 
        df1.iloc[index,i] = np.nan if len(preprocessed_text.split()) <= 2 else preprocessed_text
    i += 1
    if i >= 2: break

In [9]:
df1.isnull().values.any()

True

In [10]:
df1.isna().sum()

question     0
answer      14
dtype: int64

In [11]:
df[df1.isna().any(axis=1)].style.set_properties(subset=['question', 'answer'], **{'width': '300px'})

,question,answer
3,What are design patterns?,https://en.wikipedia.org/wiki/Software_design_pattern & https://www.geeksforgeeks.org/design-patterns-set-1-introduction/
85,"Hi Can you please clarity a doubt . Will Google AutoML eliminate the need for ML specialists in future ? Please share your views regarding this. Thanks, Pankaj",Audio replies: https://soundcloud.com/applied-ai-course/automl-vs-ml-engineer https://soundcloud.com/applied-ai-course/automated-ml
90,"1. Do you also cover the Markov models? 2. Also, do you teach Image processing as part of the course? as it will be required in few of the case studies that involve data as images 3. The case studies that are mention in the end like human activity recognition, do you only give a brief overview, or they are taught like rest of the others in detail? How do you go about teaching them? 4. We have to choose which case studies to pursue, or we can try all of them out? How exactly do you go ahead about them?",Audio reply: https://soundcloud.com/applied-ai-course/mehek-kawatra-comment/s-qVvLj
91,"Hi sir, Its better if the course includes a separate section for sequence models, where the section involves techniques like hmm, Maximum entropy markov models, conditional random fields. These are some of the good algorithms for NLP sequence labeling tasks.",Audio reply: https://soundcloud.com/applied-ai-course/hmm-and-crf
105,"Hi, 1) I am coming from Mainframe, and from last three year I am in Big Data Support project, having working knowledge of Hive, Hadoop, sqoop, etc; and understanding of spark-Scala, python, core JAVA ( from company training and self learning). Though when ever I tried for interview, since I am 12 Work exp, question come around like architecture and not like Developer. So, what interview question company will be expecting for MI, knowing that if they investing me with good money, they need more from me, as total Work of years experience. 2) for doing assignment, do we have mentor-student and student-student forum, where we can clear our doubts ?",Audio reply: https://soundcloud.com/applied-ai-course/comment-big-data/s-89yG0
120,In the assignment of performance metrics for dataset 5_a.csv in the proab score column there is all value > 0.5 so every value in predicted y will become 1. SO this is correct that every predicted value will be 1.0?There will be no 0.0 because all the score is > 0.5. I am right?,"Yes, correct."
185,"What is the effect of imbalanced data on logistic regression? I think there is none, from geometric point of view in logistic regression we are maximizing the distance of correctly classified point from the hyperplane. How can imbalanced dataset effect this?",Refer this
196,Validity of course,1year
210,"hello team, while looking at some code , I came across something like this "" yield [ a , *b] "", where a is an array and b = func()[some function]. what is the meaning of star(*) in this?",check this
221,I have around 8+ experience in automotive and auto ancillary sector .Want to learn data science .I have done lots of search and final got ur course info. Hope the person like me with no experience of IT can make a transction to Data world through your course .,happy learning


In [12]:
df = df1.dropna()

# Please drop a mail or please make a phone call type of answers

In [13]:
num_indexes, mail_indexes = list(), list()
for index, preprocessed_text in enumerate(df.iloc[:, 1].to_list()):
    # Here 25 is choosen empirically if you change it the answers to our EDA will change.
    if len(re.findall("\+91 8106-920-029", preprocessed_text)) and len(preprocessed_text.split())<25:
        num_indexes.append(index)
    elif len(re.findall("91 8106-920-029", preprocessed_text)) and len(preprocessed_text.split())<25:
        num_indexes.append(index)
    elif len(re.findall("8106-920-029", preprocessed_text)) and len(preprocessed_text.split())<25:
        num_indexes.append(index)
    elif len(re.findall("8106920029", preprocessed_text)) and len(preprocessed_text.split())<25:
        num_indexes.append(index)
        
    elif len(re.findall("\+91 6301-939-583", preprocessed_text)) and len(preprocessed_text.split())<25:
        num_indexes.append(index)
    elif len(re.findall("91 6301-939-583", preprocessed_text)) and len(preprocessed_text.split())<25:
        num_indexes.append(index)
    elif len(re.findall("6301-939-583", preprocessed_text)) and len(preprocessed_text.split())<25:
        num_indexes.append(index)
    elif len(re.findall("6301939583", preprocessed_text)) and len(preprocessed_text.split())<25:
        num_indexes.append(index)
        
    elif len(re.findall("team@appliedaicourse.com", preprocessed_text)) and len(preprocessed_text.split())<25:
        mail_indexes.append(index)

In [14]:
len(num_indexes)

39

In [15]:
df.iloc[list(set(num_indexes))].style.set_properties(subset=['question', 'answer'], **{'width': '300px'})

,question,answer
524,I m experience in electrical and planning feild in Thermal power plant. Is therr any scope here I wannted to be a data scientist. Is this ML will help me,Please call us on 91 8106-920-029
529,Hi I am from manufacturing industry backround with domain experience in materials of around 7 years. Currently I am learning Python programming. My basic qualification is BSC in Chemistry.Please let me know whether I am eligible to learn Machine Learning and chances of getting Job after completion course due to my Non IT background.,Please call us on 91 8106-920-029
537,Hi Team Would this course also cover the contents necessary in the field of Data Science. Also is Machine Learning and Data Science interrelated? And can we look for an opportunity both in the filed of as Machine Learning Engineer and Data Scientist after successful completion of this course? Many Thanks.,Please call us at 91 8106-920-029 or 91 6301-939-583 so that we can assist you better.
282,Hi Team I am currently in my final year of Engineering ECE . I wanted to make my career in data science so please tell me if this course would be a great fit and what are the chances of getting placed ?,Please call us at 6301-939-583 or 8106-920-029. We'll clarify all your doubts.
539,Sir I just want to know is there required any degree and what's the procedure for apply?,Please reach out to us to have a discussion at 91-8106-920-029 Within India or mail us at team@appliedaicourse.com
17,How many real-world projects can be done by your course in data science? How many real-world projects can be done by your course in machine learning? What is the average package of the students who got placed from your course? Are all students get the same projects or each student gets different projects? Waiting for reply. Thanks in advance,Please call us at 6301-939-583 or 8106-920-029. We'll clarify all your doubts.
418,I bought this course last year online and due to some personal issues could not complete this. As I know the validity of this course was life time but now I am not able to go though the modules as its saying that my validity has been expired. How come a lifetime course validity can be expired ? Please look into and resolve my issue so that I can start it again.,The validity of the course is 365 days. Please send an email to team@appliedaicourse.com or call us on 91 8106-920-029
546,I Just Graduated my 12th Standard CS with Maths with good marks. Can I get this course?. And what if I am not interested in Job Placements and degree. The only thing is I want to Learn AI. Please Reply Thanks in Advanced.,Please call us at 91-8106-920-029. We try to clarify all your queries over the phone.
434,facing problem to make payment.when I am trying to pay the course fee in 6 months emi option after giving my detalis and checking make payment option its taking me to the previous page.,Please send an email to team@appliedaicourse.com Otherwise you also can call us at 08106-920-029
690,since your course offer jobs .. so what is range of salary to the fresher who completed your course with your assignment with your given time duration,Kindly give us a call on 91-6301-939-583 or 91-8106-920-029 to have further and detailed discussion on it.


In [16]:
len(mail_indexes)

92

In [17]:
df.iloc[mail_indexes].style.set_properties(subset=['question', 'answer'], **{'width': '300px'})

,question,answer
12,Can I learn artificial intelligence in the field of mechatronics?,Please drop us your query at team@appliedaicourse.com. so that we can assist you better.
19,Hi Team Can you please provide the assignments at the end of each module which will directly take us to google classroom's assigment? And provide tick like for videos once the assignment is completed and graded by you.,Please drop a mail at team@appliedaicourse.com
26,and please tell how much students of yours have completed the full course in 1 year ratio wise ...i have a phobia of maths but i know programming ...so can i easily understand the algo like random forest knn nd other algo like these buy just watching your videos with 100 intrest,Please send an email to team@appliedaicourse.com
36,I also would like to know. I keep coming back to this website and thinking about enrolling BUT no access to material after 365 days throws me off. I really like the course - to me it would make sense if I do not have access to assignments and future updates. I should have continued access to video content that I use as revision or reference. All other major program providers are giving the live time access ...,Please drop a mail at team@appliedaicourse.com
111,I have been watching some of the latest live sessions and they are really good thank you for the hard word of putting them together especially in these conditions. I would like to get some feedback on my blog here : Is this the right place to post? Nikhil,Can you please write an email to the team@appliedaicourse.com about feedback to your blog?
115,While doing exception handing in try block by am appending the list but its give me empty list as resultant. And after i appended it outside try block it successfully appended so what is the reason that list cant be append in the try block. ANd other thing that i am getting ZeroDivisionerror so i am doing that when every this error occurs then i am giving that variable as 0 in exception block So we can do this. Or else how to handle ZeroDIvision error,Please mail us the code snippet that you're executing along with all problems at team@appliedaicourse.com
117,How to finish assignment,Please drop a mail to team@appliedaicourse.com regarding this query so that we can assist you better.
128,Hello Team Due to some personal i am not able to do this course and submit an assignment is it possible to extend my course validity date it will be really helpful.,Please drop a mail to team@appliedaicourse.com regarding this query so that we can assist you better.
129,I would like to enroll for this course but i have zero knowledge in programming. My complete 7 years of experience has been into client service and management. I have done B.Com and MBA Finance . Kindly advise.,Please drop a mail at team@appliedaicourse.com. We will guide you accordingly.
130,In the assignment of Tf-IDF vectorizer in task 1. give some hint to solve i have computed both tf ad idf values and after multiplying i am getting totally different answers than given in the notebook,Please drop a mail to team@appliedaicourse.com regarding this assignment query so that we can assist you better.


# Merge data from 2 dataframes

In [18]:
df = df.append(dff)
df.reset_index(inplace=True, drop=True)

In [19]:
df.style.set_properties(subset=['question', 'answer'], **{'width': '300px'})

In [20]:
df.isnull().values.any()

False

# Basic Preprocessing

In [21]:
class data_pre_processing_class():
    
    def __init__(self):
        self.preprocessed_text = " "
    
    def do_chunking(self, line):
        sents = spacy.load('en')(line)
        for person_name in [ee for ee in sents.ents if ee.label_ == 'PERSON']:
            line = re.sub(str(person_name),' ',line)
        return line
    
    def TextPreProce(self, Sentense):
        # specific
        Sentense = re.sub(r"won't", "will not", Sentense)
        Sentense = re.sub(r"can\'t", "can not", Sentense)
        # general
        Sentense = re.sub(r"n\'t", " not", Sentense)
        Sentense = re.sub(r"\'re", " are", Sentense)
        Sentense = re.sub(r"\'d", " would", Sentense)
        Sentense = re.sub(r"\'ll", " will", Sentense)
        Sentense = re.sub(r"\'t", " not", Sentense)
        Sentense = re.sub(r"\'ve", " have", Sentense)
        Sentense = re.sub(r"\'m", " am", Sentense)
        # Change repeated "????"/"....." ===> "?"/"."
        # https://stackoverflow.com/a/3878738
        Sentense = re.sub('\?\?+', '?', Sentense)
        Sentense = re.sub('\.\.+', '.', Sentense)
        return Sentense
    
    def preprocess_data_set(self, i = 0):
        while True:
            for index, self.preprocessed_text in enumerate(df.iloc[:,i].to_list()):
                # Remove the newlines, tabs. ---> https://stackoverflow.com/a/10711150
                self.preprocessed_text = re.sub('\s+',' ',self.preprocessed_text)
        
                # Decontractions, replace words like below to full words.
                self.preprocessed_text = self.TextPreProce(self.preprocessed_text)
            
                # Replace all the words except "A-Za-z0-9@.?:_/\'-" with space.
                self.preprocessed_text = re.sub('[^A-Za-z0-9@.?:_/\'-]', ' ', self.preprocessed_text)
    
                # Remove the newlines, tabs. loweer case everything, strip strings.
                df.iloc[index,i] = re.sub('\s+', ' ',self.preprocessed_text).strip().lower()
            i += 1
            if i == 2: return

In [22]:
objectt = data_pre_processing_class()
objectt.preprocess_data_set()

In [23]:
df.to_csv('preprocessed_dataset.csv', index=False)